# Plotting live data from Cyton Board using Python

## Notes:
1. This notebook is being run on a Windonws 10 Laptop, using a Python3 environment.
2. After you installed all the [dependencies](http://docs.openbci.com/OpenBCI%20Software/05-OpenBCI_Python) correctly you can start usign this tutorial
2. Start this notebook from a command prompt with sudo or as administrator.

### Importing necessary packages:
The packages used for this tutorial are in the [requirements.txt](https://github.com/andreaortuno/Plotting_OpenBCI_Cyton_Data_live/blob/master/requirements.txt) file. You can simply download that file and install the requirements by typying "pip install -r requirements.txt" on the command line.

If you are having trouble with the plugin_interface you might need to use [this workaround](https://github.com/OpenBCI/OpenBCI_Python/issues/91#issuecomment-412817422)

#### First let's import the necessary packages


In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import collections
%matplotlib inline

import sys; sys.path.append('..') # help python find cyton.py relative to scripts folder
from openbci import cyton as bci
import logging
import time
import datetime as dt
from scipy import signal

#### Now let's try to run the [test code](https://github.com/OpenBCI/OpenBCI_Python/blob/master/scripts/test.py) from the OpenBCI-Python repo

We are doing this to make sure all the dependencies are installed correctly. When working this code should print the raw data of the board. If everything is working correctly you can press crtl+C to end the script.

In [ ]:
def printData(sample):
    # os.system('clear')
    print("----------------")
    print("%f" % (sample.id))
    print(sample.channel_data)
    print(sample.aux_data)
    print("----------------")

if __name__ == '__main__':
    port = 'COM5'
    baud = 115200
    logging.basicConfig(filename="test.log", format='%(asctime)s -%(levelname)s : %(message)s', level=logging.DEBUG)
    logging.info('--------------LOG START ------------')
    board = bci.OpenBCICyton(port=port, scaled_output=False, log=True)
    print("Board Instantiated")
    board.ser.write(str.encode('v'))
    time.sleep(10)
    board.start_streaming(printData)
    board.print_bytes_in()

#### Now we can add code to be able to plot the data

We will define a simple funtion live_plot to plot the raw data onto the notebook. We also need to define another fuction, let's call it "data_process", that takes in the raw data from the board, stores it in a global variable and allows us to dynamically change the plot as we get more signals and calls on the live_plot function. For this example let's plot only the first 5 channels.

In [ ]:
def live_plot(sample, figsize=(17,16)):
    data = np.array(sample)
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    color = 'rgbycmkr'
    labels = ['Channel 1', 'Channel 2', 'Channel 3', 'Channel 4', 'Channel 5', 'Channel 6', 'Channel 7', 'Channel 8']
    
    try:
        for j in range(5):
            num = int('81'+str(j+1))
            plt.subplot(num)
            if j == 0:
                plt.title("EEG Data")
            plt.plot(data[-40:][:,j], label=labels[j], scaley=True, color=color[j])
            plt.autoscale(enable=True, axis='y', tight=None)
        
    except:
        for j in range(5):
            num = int('81'+str(j+1))
            plt.subplot(num)
            if j == 0:
                plt.title("EEG Data")
            plt.plot(data[-40:][:,j], label=labels[j], scaley=True, color=color[j])
            plt.autoscale(enable=True, axis='y', tight=None)
            
    plt.xlabel('time')
    plt.legend(loc='center left') # the plot evolves to the right
    plt.show();
    
    
bci_data = []

def data_process(sample):
    global bci_data
    Scale_Factor= (4.5/ 24) /(2^23 - 1) # specific for Cyton
    data = [i * Scale_Factor for i in sample.channel_data]
    bci_data.append(data)
    live_plot(bci_data)

Now let's run the code we just added.

In [ ]:
if __name__ == '__main__':
    port = 'COM5'
    baud = 115200
    logging.basicConfig(filename="test.log", format='%(asctime)s -%(levelname)s : %(message)s', level=logging.DEBUG)
    logging.info('--------------LOG START ------------')
    board = bci.OpenBCICyton(port=port, scaled_output=False, log=True)
    print("Board Instantiated")
    board.ser.write(str.encode('v'))
    time.sleep(10)
    board.start_streaming(data_process)
    board.print_bytes_in()

The graph is a bit slow, but this is a great way to start to understand how to use the OpenBCI data for your application.

#### Signal Processing:

The next steps on your OpenBCI Python journey would be to do some signal processing. [Here](https://github.com/J77M/openbciGui_filter_test/blob/master/gui_saved_data_filter.ipynb) is a good example of some of the filters you can use with your OpenBCI data.